**Connect google drive**

In [25]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'https:  vm.tikt.txt'	       IMG_20170817_182819_HDR.jpg
 IMG_20170731_120800.jpg       IMG_20170817_182824_HDR.jpg
 IMG_20170804_113120.jpg       IMG_20170817_182828_HDR.jpg
 IMG_20170804_113124.jpg       IMG_20170817_182832_HDR.jpg
 IMG_20170804_201647_HHT.jpg   IMG_20170817_182836_HDR.jpg
 IMG_20170804_201653_HHT.jpg   IMG_20170817_182843_HDR.jpg
 IMG_20170804_201659_HHT.jpg   IMG_20170817_182848_HDR.jpg
 IMG_20170804_201708_HHT.jpg   IMG_20170817_182911_HDR.jpg
 IMG_20170804_201710_HHT.jpg   IMG_20170817_182914_HDR.jpg
 IMG_20170804_201711_HHT.jpg   IMG_20170817_182917_HDR.jpg
 IMG_20170804_201732.jpg       IMG_20170817_182927_HDR.jpg
 IMG_20170804_201747.jpg       IMG_20170817_182932_HDR.jpg
 IMG_20170804_201752.jpg       IMG_20170817_182946_HDR.jpg
 IMG_20170804_201756.jpg       IMG_2

**1) Clone, configure & compile Darknet**



In [21]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [22]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

[Errno 20] Not a directory: 'darknet'
/content/darknet


In [23]:
# Compile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1139:10: note: ...thi

**2) Configure yolov3.cfg file**

In [9]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [10]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [11]:
!echo -e 'person\nperson_with_ladder\nperson_without_ladder' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [12]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [13]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/280.JPG        
  inflating: data/obj/280.txt        
  inflating: data/obj/281.JPG        
  inflating: data/obj/281.txt        
  inflating: data/obj/282.JPG        
  inflating: data/obj/282.txt        
  inflating: data/obj/283.JPG        
  inflating: data/obj/283.txt        
  inflating: data/obj/classes.txt    
  inflating: data/obj/1.jpg          
  inflating: data/obj/1.txt          
  inflating: data/obj/2.JPG          
  inflating: data/obj/2.txt          
  inflating: data/obj/3.JPG          
  inflating: data/obj/3.txt          
  inflating: data/obj/4.JPG          
  inflating: data/obj/4.txt          
  inflating: data/obj/5.JPG          
  inflating: data/obj/5.txt          
  inflating: data/obj/9.JPG          
  inflating: data/obj/9.txt          
  inflating: data/obj/10.JPG         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.JPG         
  inflating: data/obj/11.txt         
  inflating: 

**6) Create train.txt file**

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [15]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-28 01:08:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  91.3MB/s    in 1.7s    

2021-05-28 01:08:19 (91.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [26]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
!./darknet/darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show
#!./darknet/darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights  -dont_show 

/bin/bash: ./content/darknet/darknet: No such file or directory
